# Sobol Sensitivity Indices

By analysis of variance (ANOVA) or [Sobol decomposition](https://www.sciencedirect.com/science/article/abs/pii/S0378475400002706), a model function is decomposed as,
$$
\begin{equation}
f(\mathbf{q}) = 
f_0 +\sum_{i=1}^p f_i(q_i) + \sum_{1\leq i<j\leq p} f_{ij}(q_i,q_j)+\cdots\,,
\label{eq:anova_f}\tag{1}
\end{equation}
$$

where, $f_0$ is the mean of $f(\mathbf{q})$, $f_i(q_i)$ specify the contribution of each parameter, $f_{ij}(q_i,q_j)$ denote effects of
interaction between each pair of parameters, and so on for other interactions. 
These contributors are defined as,
$$
\begin{eqnarray*}
f_0 &=& \mathbb{E}_\mathbf{q}[f(\mathbf{q})] \,, \\
f_i(q_i) &=&\mathbb{E}_\mathbf{q}[f(\mathbf{q})|q_i] - f_0 \,, \\
f_{ij}(q_{i},q_j) &=& \mathbb{E}_\mathbf{q}[f(\mathbf{q})|q_i,q_j] -f_i(q_i) -f_j(q_j) - f_0 \,. 
\end{eqnarray*}
$$

Here, $\mathbb{E}_\mathbf{q}[f(\mathbf{q})|q_i]$, for instance, denotes the expected value of $f(\mathbf{q})$ conditioned on fixed values of $q_i$. 
Similar to Eq. \eqref{eq:anova_f}, the total variance of $f(\mathbf{q})$, denoted by $D$, is decomposed as, 
$$
\begin{equation}
\mathbb{V}_\mathbf{q}[f(\mathbf{q})] = D=\sum_{i=1}^p D_i + \sum_{1\leq i<j\leq p} D_{ij} + \cdots \,,
\end{equation}
$$

where, $D_i=\mathbb{V}_\mathbf{q}[f_i(q_i)]$, $D_{ij}=\mathbb{V}_\mathbf{q}[f_{ij}(q_i,q_j)]$, and  so on. 
The main Sobol indices are eventually defined as the contribution of
each of $D_i$, $D_{ij}$, ... in the total variance $D$:
$$
\begin{equation}
S_i=D_i/D\,,\quad 
S_{ij}=D_{ij}/D \,,\, \ldots \,, \quad i,j=1,2,\cdots,p 
\label{eq:sobol} \tag{2}
\end{equation}
$$

### Example 1: Sobol indices for 2D parameter space

Consider the following analytical function, 
$$
f(\mathbf{q}) = q_1^2+q_1 q_2
$$

where $\mathbf{q}=\{q_1,q_2\}$, and both parameters are random with uniform distributions, i.e., $q_1\sim \mathcal{U}[\mathbb{Q}_1]$ and $q_2\sim \mathcal{U}[\mathbb{Q}_2]$. 
The aim is to compute the main Sobol indices using `UQit` and validate the values with the analytical data.